An example of creating a subclass of HyperImg and working with it

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import tifffile as tiff

import hyper_img as hyper

Pathes and constants

In [ ]:
IMG_PATH = '/home/igor/projects/Seed/data/golozernost'
CALIBR_WHITE_PATH = IMG_PATH + '/2022_12_21_session_001_004_snapshot_cube.tiff'
CALIBR_BLACK_PATH = IMG_PATH + '/2022_12_21_session_001_005_snapshot_cube.tiff'
TARGET_VARIABLE = 'Голозерность/Пленчатость'

Example of HyperImg subclass

In [ ]:
import hyper_img as hyper

class GolozernostHyperImg(hyper.HyperImg):
    
    def _get_tiff(self) -> np.ndarray:
        img = tiff.imread(self.path)
        bl_img = tiff.imread(CALIBR_BLACK_PATH)
        new_img = np.where(bl_img > img, 0, img - bl_img)
        wh_img = tiff.imread(CALIBR_WHITE_PATH)
        try:
            return new_img /(wh_img - bl_img)
        except ZeroDivisionError:
            raise ZeroDivisionError('black image is equal to white in one component')

    
    def _get_target_varible(self) -> str:
        if not set(IMG_PATH.split('/')).issubset(set(self.path.split('.')[0].split('/'))):
            raise NameError('Error in path')
        
        if 'голозерные' in self.path:
            return 'голозерные'
        
        if 'пленчатые' in self.path:
            return 'пленчатые'
        
        return 'калибровка'

Get a list of GolozernostHyperImg

In [ ]:
img_lst: list[GolozernostHyperImg] = hyper.get_list_hyper_img(path=IMG_PATH, 
                                                              class_name=GolozernostHyperImg,
                                                              filter=lambda x: x != 'калибровка',
                                                              threshold_value=25,
                                                              target_varible_name=TARGET_VARIABLE,
                                                              black_calibration_img_name=CALIBR_BLACK_PATH.split('/')[-1],
                                                              white_calibration_img_name=CALIBR_WHITE_PATH.split('/')[-1])

Example of elements and some main operation with them

In [ ]:
img_lst[0]

In [ ]:
img_lst[1]

In [ ]:
img_lst[0] - img_lst[1]

In [ ]:
img_lst[0] + img_lst[1]

Charts

In [ ]:
hyper.get_medians_wavelenght_graph(img_lst)

In [ ]:
hyper.get_mean_diff_and_confident_interval_graph(img_lst, 'пленчатые', 'голозерные', 0.95)

In [ ]:
hyper.get_2_pca_graph(img_lst)

In [ ]:
hyper.get_umap_graph(img_lst, n_neighbors=3)

Some examples of getting useful pd.DataFrame (see more in get_data_funcs.py)

In [ ]:
df = hyper.get_df_graphics_medians_wavelenght(img_lst)
df.head()

In [ ]:
plt.figure(figsize=(7,5))
plt.grid()
sns.lineplot(data=hyper.get_df_graphics_medians_wavelenght(img_lst), 
             x='Wavelength', y='Median', hue=TARGET_VARIABLE)

In [ ]:
df = hyper.get_df_medians(img_lst)
df.head()

In [ ]:
df[TARGET_VARIABLE].hist()

In [ ]:
df, explained_variance = hyper.get_df_2_pca_and_explained_variance(img_lst)
df.head()

Create annotation

In [ ]:
hyper.create_table_annotation_df(img_lst)